# Setup

In [2]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np

In [3]:
sys.path.append(os.path.abspath("../../.."))

In [4]:
from src.experiment.helpers.variables import dataset_root_dir
from src.experiment.helpers.utils import get_sorted_class_percentages_label_encoded

In [5]:
NAME = "gas-drift"
CLASSIFICATION_TYPE = "multiclass"

In [6]:
DATASET_DIR = dataset_root_dir / CLASSIFICATION_TYPE / NAME
print(DATASET_DIR)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\multiclass\gas-drift


In [7]:
balanced_name = NAME + "_balanced"
DATASET_DIR_BALANCED = dataset_root_dir / CLASSIFICATION_TYPE / balanced_name
print(DATASET_DIR_BALANCED)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\multiclass\gas-drift_balanced


In [8]:
imbalanced_name = NAME + "_imbalanced"
DATASET_DIR_IMBALANCED = dataset_root_dir / CLASSIFICATION_TYPE / imbalanced_name
print(DATASET_DIR_IMBALANCED)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\multiclass\gas-drift_imbalanced


# Dataset loading

In [9]:
X = pd.read_csv(DATASET_DIR / 'X.csv')
y = pd.read_csv(DATASET_DIR / 'y.csv')

In [10]:
X

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128
0,12285.6582,4.076635,4.842317,7.509393,10.822436,-1.312657,-1.853717,-6.924985,11800.9233,4.483500,...,-0.747592,-3.440387,1784.5324,1.907000,1.729200,4.881194,8.623828,-0.314110,-0.661556,-3.521663
1,-35.6889,0.993944,0.166099,0.489363,3.484663,-0.130298,-0.528364,-3.735347,266.4145,1.053988,...,-0.622342,-4.482534,904.9898,1.433707,1.068069,2.532958,5.369720,-0.183779,-0.534087,-4.635975
2,63927.2217,14.956941,19.971376,29.188512,33.291320,-10.433776,-16.062245,-49.490143,57405.8483,15.613843,...,-7.762551,-25.150090,14585.7879,8.189021,6.099452,12.127991,15.709651,-3.887082,-6.731473,-19.326895
3,2992.9019,1.380553,0.808910,1.288259,4.660135,-0.755903,-1.120470,-4.075213,4301.4033,1.652701,...,-3.244969,-7.215792,6044.5554,3.488295,2.662288,5.938297,8.544508,-1.567322,-2.701235,-6.472439
4,57524.7812,11.912566,14.631496,19.809240,23.715868,-9.084750,-11.770585,-39.234003,50051.0703,11.732548,...,-5.093413,-10.248794,10580.1006,5.752675,3.880740,8.545897,11.831716,-2.655521,-4.312744,-8.510591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13905,13384.8262,2.820931,4.007378,6.618008,11.386095,-2.142994,-3.110327,-11.296786,19013.4575,4.050907,...,-3.131500,-8.012662,7548.3501,2.377836,3.400734,8.571174,11.555882,-1.617656,-2.457614,-6.226359
13906,13382.9619,2.825174,4.010915,6.483989,10.356127,-2.156512,-3.179563,-10.184803,19034.2495,4.066463,...,-2.919325,-6.943002,7510.4946,2.364505,3.401381,8.512949,12.149638,-1.613554,-2.493870,-6.859804
13907,13336.8725,2.822288,3.980818,6.487103,10.936979,-2.146688,-3.273109,-11.067489,18997.7222,4.055524,...,-2.924455,-7.777268,7530.0010,2.369898,3.400592,8.494436,11.839013,-1.612525,-2.504918,-6.263872
13908,13351.1318,2.824358,3.987819,6.554427,11.331002,-2.143651,-3.257854,-11.795109,19035.9926,4.071607,...,-3.141852,-6.890286,7599.0201,2.391834,3.358804,8.457260,11.297346,-1.606879,-2.438701,-6.044784


In [11]:
y

,Class
0,4
1,3
2,4
3,3
4,4
...,...
13905,6
13906,6
13907,6
13908,6


# Cut classes to match balanced dataset

In [12]:
y_balanced = pd.read_csv(DATASET_DIR_BALANCED / 'y.csv')
y_balanced

,Class
0,3
1,3
2,3
3,3
4,3
...,...
1620,6
1621,6
1622,6
1623,6


In [13]:
# Get the set of classes present in y_balanced
classes_in_balanced = set(y_balanced['Class'].unique())
print(f"Classes in balanced dataset: {classes_in_balanced}")

# Create mask for samples in y that are present in y_balanced
mask = y['Class'].isin(classes_in_balanced)

# # Filter X and y
X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

Classes in balanced dataset: {np.int64(1), np.int64(2), np.int64(3), np.int64(5), np.int64(6)}


# Dataset info

In [17]:
# number of samples
len(X)

11974

In [18]:
# number of classes (number of unique numbers in y)
len(np.unique(y.values))

5

In [19]:
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
print(class_percentages_sorted)

5    25.129447
2    24.436279
1    21.421413
6    15.308168
3    13.704694
Name: proportion, dtype: float64


# Functions

In [21]:
def get_least_represented_class_names(class_percentages_sorted, percent):
    num_classes = len(class_percentages_sorted)
    bottom_percent_count = int(np.ceil(num_classes * percent))
    least_represented_classes = class_percentages_sorted.tail(bottom_percent_count).index.tolist()
    return least_represented_classes

In [22]:
def get_most_represented_class_names(class_percentages_sorted, percent):
    num_classes = len(class_percentages_sorted)
    top_percent_count = int(np.ceil(num_classes * percent))
    most_represented_classes = class_percentages_sorted.head(top_percent_count).index.tolist()
    return most_represented_classes

In [23]:
def show_summary(X, y):
    print("Number of samples:", len(X))
    print("Number of classes:", len(np.unique(y.values)))

    class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
    
    highest_class_percentage = class_percentages_sorted.iloc[0]
    lowest_class_percentage = class_percentages_sorted.iloc[-1]
    
    if lowest_class_percentage != 0:
        ratio = highest_class_percentage / lowest_class_percentage
        print(f"Times higher: {round(ratio, 2)}")
    else:
        ratio = np.nan
        print("The lowest class percentage is 0, cannot calculate ratio.")
        
    average_class_percentage = class_percentages_sorted.mean()
    class_percentages_std = class_percentages_sorted.std()

    cov = class_percentages_std / average_class_percentage
    print(f"CoV: {round(cov, 3)}")
    
    print(class_percentages_sorted)
    
    return class_percentages_sorted

In [24]:
def remove_class(X, y, chosen_class_index):
    # If DataFrame, select the first column
    if isinstance(y, pd.DataFrame):
        y_col = y.columns[0]
        y_series = y[y_col]
    else:
        y_series = y


    print(f"Removing class: {chosen_class_index}")
    
    # Create mask from 1D series
    mask = y_series != chosen_class_index

    # Filter X and y using 1D mask
    return X.loc[mask], y.loc[mask]


In [25]:
# removes SAMPLES
def remove_few_samples_of_class(X, y, class_index, step):
    indices_to_remove = y[y.values.flatten() == class_index].index[:step]
    X_trimmed = X.drop(indices_to_remove)
    y_trimmed = y.drop(indices_to_remove)
    
    return X_trimmed, y_trimmed

# Dataset trimming

In [23]:
some_classes = get_most_represented_class_names(class_percentages_sorted, 0.8)
some_classes

[1, 0]

In [ ]:
# remove top class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[0]

X,y = remove_class(X, y, chosen_class)
class_percentages_sorted = show_summary(X, y)

Removing class: 1
Number of samples: 600
Number of classes: 1
0    100.0
Name: proportion, dtype: float64


In [20]:
# remove bottom class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[-3]

X,y = remove_class(X, y, chosen_class)
class_percentages_sorted = show_summary(X, y)

Removing class: 4
Number of samples: 11974
Number of classes: 5
Times higher: 1.83
CoV: 0.262
5    25.129447
2    24.436279
1    21.421413
6    15.308168
3    13.704694
Name: proportion, dtype: float64


In [219]:
# remove samples from top class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[0]

X,y = remove_few_samples_of_class(X, y, chosen_class, step=100)
class_percentages_sorted = show_summary(X, y)

Number of samples: 4173
Number of classes: 4
Times higher: 5.01
CoV: 0.497
6    31.943446
5    31.368320
1    30.313923
3     6.374311
Name: proportion, dtype: float64


In [141]:
# remove samples from bottom class
class_percentages_sorted = get_sorted_class_percentages_label_encoded(y)
chosen_class = class_percentages_sorted.index[-1]

X,y = remove_few_samples_of_class(X, y, chosen_class, step=1)
class_percentages_sorted = show_summary(X, y)

Number of samples: 1620
Number of classes: 5
Times higher: 5.02
CoV: 0.494
5    34.691358
3    20.308642
6    20.000000
2    18.086420
1     6.913580
Name: proportion, dtype: float64


# Trimmed dataset - info

In [125]:
class_percentages_sorted = show_summary(X, y)

Number of samples: 1634
Number of classes: 5
Times higher: 4.98
CoV: 0.5
5    35.067319
3    20.134639
6    19.828641
2    17.931457
1     7.037944
Name: proportion, dtype: float64


# Save

### Balanced

In [115]:
DATASET_DIR_BALANCED.mkdir(parents=True, exist_ok=True)
X.to_csv(DATASET_DIR_BALANCED / "X.csv", index=False)
y.to_csv(DATASET_DIR_BALANCED / "y.csv", index=False)

In [116]:
class_percentages_sorted.to_csv(DATASET_DIR_BALANCED / "class_percentages.csv", header=False)

### Imbalanced

In [142]:
DATASET_DIR_IMBALANCED.mkdir(parents=True, exist_ok=True)
X.to_csv(DATASET_DIR_IMBALANCED / "X.csv", index=False)
y.to_csv(DATASET_DIR_IMBALANCED / "y.csv", index=False)

In [143]:
class_percentages_sorted.to_csv(DATASET_DIR_IMBALANCED / "class_percentages.csv", header=False)